# Alcohol Consumption in Russia

![Alcoholic Beverages in Russia](images/drinks.png)
            Source: [The Russian alcohol market: a heady cocktail](http://www.food-exhibitions.com/Market-Insights/Russia/The-Russian-alcohol-market)

## Project Motivation

A fictitious company owns a chain of stores across Russia that sell a variety of types of alcohol. The company recently ran a wine promotion in Saint Petersburg that was very successful. Due to the cost to the business, it isn’t possible to run the promotion in all regions. The marketing team would like to target 10 other regions that have similar buying habits to Saint Petersburg where they would expect the promotion to be similarly successful and need help determining which regions they should select.

![Regions in Russia](images/regions.png)
        Source: [Outline of Russia](https://en.wikipedia.org/wiki/Outline_of_Russia)
        
This project aims to use machine learning algorithm to recommend, at least 10 regions with alcohol buying habits similar to Saint Petersburg. 

## The Dataset

The data used in this project is obtained from [Datacamp's Career Hub repository](https://github.com/datacamp/careerhub-data) on GitHub. It contains 7 variables as see in the description below:

![Description of dataset](images/data_description.png)

## Analysis Plan

Based on the ask of the project, the problem is best solved using an unsupervied machine learning algorithm that could best cluster regions based on wine sales in Saint Petersburg. Selection of this algorithm will be done in subsequent sections.

The following steps will be followed:

- Perform Exploratory Data Analysis to identify patters and draw insights from the data.
- Select a suitable unsupervised machine learning algorithm based on problem to solve and information from the exploratory data analysis.
- Discuss model performance.

### Exploratory Data Analysis

This section will explore the data to discover trends and insights. It will be done by creating plots of features against their values. The following steps will be implemented:

- Read data
- Check for data quality issues.
- Clean and transform data into a suitable format for exploration.
- Data Visualization to observe patterns and trends.

In [3]:
# import system and exploratory analysis modules
import platform; print(platform.platform())
import sys; print("Python", sys.version)
import numpy as np; print("Numpy", np.__version__)
import matplotlib
import matplotlib.pyplot as plt; print("Matplotlib", matplotlib.__version__)
import pandas as pd; print("Pandas", pd.__version__)
import seaborn as sns; print("Seaborn", sns.__version__)
import scipy; print("Scipy", scipy.__version__)
import sklearn; print("Scikit -Learn", sklearn.__version__)

Windows-10-10.0.19041-SP0
Python 3.6.12 |Anaconda, Inc.| (default, Sep  9 2020, 00:29:25) [MSC v.1916 64 bit (AMD64)]
Numpy 1.19.2
Matplotlib 3.3.2
Pandas 1.1.5
Seaborn 0.11.1
Scipy 1.5.2
Scikit -Learn 0.23.2


### Read and check data for quality issues

A function us created to do the following:
- Read the data
- Drop duplicates
- Create a list of feature names
- Create a containing data types and nulls.

In [17]:
# function to read data, check for nulls and drop duplicates
def read_data(data_path):
    # read data
    print("Reading Alcohol Consumption in Russia dataset\n")
    df = pd.read_csv(data_path)
    # make a copy of dataframe
    print("Making a copy of the dataframe\n")
    df_1 = df.copy()
    # drop duplicates
    df_final = df_1.drop_duplicates()
    # extract feature names
    df_cols = df_final.columns.tolist()
    print("Data consists of:\n")
    print("...........................\n")
    print("Rows: {}\n".format(len(df_final)))
    print("Columns: {}\n".format(len(df_cols)))
    print("...........................\n")
    # empty list to hold data types, non nulss count, nulss count, percentage of nulls in a column,\
    # percentage of column nulls in datafram
    data_types = []
    non_nulls = []
    nulls = []
    null_column_percent = []
    null_df_percent = []
    
    # loop through columns and capture the variables above
    print("Extracting count and percentages of nulls and non nulls")
    for col in df_cols:
        
        # extract null count
        null_count = df_final[col].isna().sum()
        nulls.append(null_count)
        
        # extract non null count
        non_null_count = len(df_final) - null_count
        non_nulls.append(non_null_count)
        
        # extract % of null in column
        col_null_perc = 100 * null_count/len(df_final)
        null_column_percent.append(col_null_perc)
        
        # extract % of nulls out of total nulls in dataframe
        df_null_perc = 100 * null_count/df_final.isna().sum().sum()
        null_df_percent.append(df_null_perc)
        
        # capture data types
        data_types.append(df_final[col].dtypes) 
        
    # create zipped list with column names, data_types, nulls and non nulls
    lst_data = list(zip(df_cols, data_types, non_nulls, nulls, null_column_percent, null_df_percent))
    # create dataframe of zipped list
    df_zipped = pd.DataFrame(lst_data, columns = ['Feature', 'DataType', 'CountOfNonNulls', 'CountOfNulls',\
                                                 'PercentOfNullsIinColumn', 'PercentOfNullsInData'])
    return df_final, df_cols, df_zipped

In [18]:
path = 'alcohol-consumption-in-russia.csv'
df, features, df_QA = read_data(path)

Reading Alcohol Consumption in Russia dataset

Making a copy of the dataframe

Data consists of:

...........................

Rows: 1615

Columns: 7

...........................

Extracting count and percentages of nulls and non nulls


In [19]:
# inspect first 5 rows of data
df.head()

,year,region,wine,beer,vodka,champagne,brandy
0,1998,Republic of Adygea,1.9,8.8,3.4,0.3,0.1
1,1998,Altai Krai,3.3,19.2,11.3,1.1,0.1
2,1998,Amur Oblast,2.1,21.2,17.3,0.7,0.4
3,1998,Arkhangelsk Oblast,4.3,10.6,11.7,0.4,0.3
4,1998,Astrakhan Oblast,2.9,18.0,9.5,0.8,0.2


In [20]:
# check for quality issues in data
df_QA

,Feature,DataType,CountOfNonNulls,CountOfNulls,PercentOfNullsIinColumn,PercentOfNullsInData
0,year,int64,1615,0,0.000000,0.000000
1,region,object,1615,0,0.000000,0.000000
2,wine,float64,1552,63,3.900929,20.257235
3,beer,float64,1557,58,3.591331,18.649518
4,vodka,float64,1554,61,3.777090,19.614148
5,champagne,float64,1552,63,3.900929,20.257235
6,brandy,float64,1549,66,4.086687,21.221865


In [24]:
# summary statistics for numerical features
df.describe()

,year,wine,beer,vodka,champagne,brandy
count,1615.000000,1552.000000,1557.000000,1554.000000,1552.000000,1549.000000
mean,2007.000000,5.628144,51.260148,11.818694,1.313177,0.526998
std,5.478922,2.813208,25.372821,5.128806,0.797956,0.400201
min,1998.000000,0.100000,0.400000,0.050000,0.100000,0.000000
25%,2002.000000,3.575000,32.400000,8.300000,0.800000,0.200000
50%,2007.000000,5.400000,49.970000,11.500000,1.200000,0.400000
75%,2012.000000,7.377500,67.400000,15.000000,1.665000,0.700000
max,2016.000000,18.100000,207.300000,40.600000,5.560000,2.300000


In [25]:
# numerical data
df.describe(exclude = 'number')

,region
count,1615
unique,85
top,Kamchatka Krai
freq,19


As can be observed above, the dataset consists of 1615 rows and 7 columns. Region is the only column of type object. Also, we can tell that there are missing values in all the columns containing data of the alcoholic beverages. Noteably, Brandy has the most missing values, about 21.2% of total nulls in data. We will handle missing values in the cleaning and transformation sections. 

The summary statistics gave us an overview of basic statistical properties. We can tell that the data contains sales per capita records from the 2002 to 2016. There are 85 regions covered and Kamchatka Krai is the most frequent among region values.

### Clean and Transform Data

The data is relatively clean based on our observation from inspection section. We will handle missing values by imputing the mean of values in each beverage column and also, strip leading and trailing spaces in region column.

A function is created to perform the cleaning and transformation processes stated above.

In [26]:
# function to clean and transform data
def clean_and_transform(data, cols):
    for col in cols:
        if data[col].dtypes == 'int64' or data[col].dtypes == 'int32' or data[col].dtypes == 'float64':
            data[col] = data[col].fillna(data[col].mean())
        elif data[col].dtypes == 'object':
            data[col] = data[col].apply(lambda x: x.strip())
    return data

In [27]:
# clean and transform data
df_clean = clean_and_transform(df, features)

In [ ]:
df